In [125]:
from glob import glob
import rasterio
from rasterio.crs import CRS
import os
import pandas as pd
pd.set_option('display.max_colwidth', None)
import geopandas as gpd
from dicttoxml import dicttoxml
from xml.dom.minidom import parseString
import re

In [ ]:
delivery_name = 'Kangaroo_Island2020'
# delivery_name = 'KangarooIsland_AdditionalScene'
outputs = 'NVIS_8L_fm_mape_r1_KI2020'

# delivery_name = 'KangarooIsland_2025'
# outputs = 'NVIS_8L_fm_mape_r1_KI2025'

# delivery_name = 'Billabong'
# outputs = 'NVIS_8L_fm_mape_r1_Billabong'

# delivery_name = 'MulgaLands'
# outputs = 'NVIS_8L_fm_mape_r1_MulgaLands'

# delivery_name = 'Tenterfield'
# outputs = 'NVIS_8L_fm_mape_r1_Tenterfield'

nn_outputs_path = f'/mnt/datapool1/datapool1/datasets/nvis_outputs/{outputs}/raw'
polygon_path = f'/mnt/datapool1/datapool1/datasets/nvis_outputs/{outputs}/clipping_polygons'
output_path = f'/mnt/datapool1/datapool1/datasets/nvis_outputs/{outputs}/clipped'
merged_path = f'/mnt/datapool1/datapool1/datasets/nvis_outputs/{outputs}/merged'
processing_summary = glob(f'/mnt/datapool2/Archive/EO_IMAGERY/raw/aoi/{delivery_name}/**/01_Raw/processing_summary.csv', recursive=True)[0]

polygons = glob('/mnt/datapool1/datapool1/datasets/nn_datasets/polygons/KI2020/*.geojson')
nn_datasets = glob('/mnt/datapool1/datapool1/datasets/nn_datasets/NVIS/**/*.hdf5', recursive=True)

In [106]:
def get_geospatial_metadata(merged_tif):
    metadata = {}

    with rasterio.open(merged_tif) as src:
        metadata['driver'] = src.driver
        metadata['count'] = src.count
        metadata['width'] = src.width
        metadata['height'] = src.height
        metadata['dtype'] = src.dtypes[0]
        metadata['nodata'] = src.nodata

        metadata['crs'] = src.crs.to_string()

        metadata['bounds'] = {
            'left': src.bounds.left,
            'bottom': src.bounds.bottom,
            'right': src.bounds.right,
            'top': src.bounds.top
        }

        metadata['resolution'] = {
            'x': abs(src.res[0]),
            'y': abs(src.res[1])
        }

        metadata['transform'] = list(src.transform)

        width_meters = (src.bounds.right - src.bounds.left)
        height_meters = (src.bounds.top - src.bounds.bottom)
        metadata['spatial_extent'] = {
            'width_meters': width_meters,
            'height_meters': height_meters,
            'area_square_meters': width_meters * height_meters
        }

        # metadata['bands'] = [
        #     {'band_number': 1, 'height_range': 'below_0.5m', 'min': },
        #     {'band_number': 2, 'height_range': '0.5-1m'},
        #     {'band_number': 3, 'height_range': '1-2m'},
        #     {'band_number': 4, 'height_range': 'above_2m'},
        #     {'band_number': 5, 'height_range': 'below_3m'},
        #     {'band_number': 6, 'height_range': 'below_10m'},
        #     {'band_number': 7, 'height_range': '10-30m'},
        #     {'band_number': 8, 'height_range': 'above_30m'}
        # ]
        metadata['bands'] = []
        for band_num in range(1, src.count + 1):
            band_data = src.read(band_num, masked=True)
            
            # Calculate statistics, ignoring nodata values
            band_info = {
                'band_number': band_num,
                'height_range': {
                    1: 'below_0.5m',
                    2: '0.5-1m', 
                    3: '1-2m',
                    4: 'above_2m',
                    5: 'below_3m',
                    6: 'below_10m',
                    7: '10-30m',
                    8: 'above_30m'
                }.get(band_num, f'band_{band_num}'),
                'min': float(band_data.min()) if not band_data.mask.all() else None,
                'max': float(band_data.max()) if not band_data.mask.all() else None,
                'mean': float(band_data.mean()) if not band_data.mask.all() else None,
                'std': float(band_data.std()) if not band_data.mask.all() else None,
                'dtype': str(band_data.dtype),
                'valid_pixels': int((~band_data.mask).sum()),
                'total_pixels': int(band_data.size)
            }
            metadata['bands'].append(band_info)

        metadata['units'] = 'cover'

        # Additional technical metadata
        metadata['tiled'] = src.is_tiled
        if src.is_tiled:
            metadata['block_size'] = src.block_shapes[0]  # (height, width) of blocks
        
        # Get compression info from profile if available
        try:
            metadata['compression'] = src.compression.name if src.compression else None
        except:
            metadata['compression'] = None
            
        # Tags (additional metadata stored in the file)
        # metadata['tags'] = dict(src.tags())

    return metadata

In [10]:
print(nn_datasets[0])
print(nn_datasets[0].split('/')[7])


/mnt/datapool1/datapool1/datasets/nn_datasets/NVIS/train/250618_K_ISLAND_2_RECORD_Record008_7855_067_chm_blur20_cc8_layer/200130_0127Q6_PH6_LN_dn_s4_i5__LGN_town_ngc5_ch_LC_go2_sb_st_wg_ko_bound_train_test/KangarooIsland_2025_JL1KF02B06_0018_20250210_1380085_COMBINED_Ortho_Merged.hdf5
train


In [40]:
def get_polygon_area(file, decimals=6, clip_to=None, dissolve=False):
    """Calculate area of polygons in hectares, optionally clipped to a boundary
    
    Args:
        file (str): Path to vector file or geodataframe
        decimals (int): Number of decimal places to round to
        clip_to (GeoDataFrame, optional): GeoDataFrame to clip geometries to
        
    Returns:
        str: Area in hectares as string
    """
    if isinstance(file, str):
        # Check if file exists
        if not os.path.exists(file):
            raise FileNotFoundError(f"File {file} does not exist.")
        gdf = gpd.read_file(file)
    elif isinstance(file, gpd.GeoDataFrame):
        gdf = file
    else:
        raise ValueError("Input must be a file path or a GeoDataFrame.")
    # gdf['geometry'] = gdf['geometry'].apply(lambda geom: geometrycollection_to_multipolygon(geom))
    if clip_to is not None:
        # Handle clip_to as either string file path or GeoDataFrame
        if isinstance(clip_to, str):
            clip_gdf = gpd.read_file(clip_to)
        else:
            clip_gdf = clip_to

        # Ensure same CRS
        if gdf.crs != clip_gdf.crs:
            clip_gdf = clip_gdf.to_crs(gdf.crs)
        
        # Perform clip
        gdf = gpd.clip(gdf, clip_gdf)

    if dissolve:
        gdf = gdf.dissolve()
    return round(gdf.geometry.area.sum() / 10000, decimals)

def get_train_test_areas(train_test_polygons):
    """
    Get the train and test areas from the train_test_polygons files
    """
    train_test_areas = {'train_area': 0, 'test_area': 0}
    for file in train_test_polygons:
        file_gdf = gpd.read_file(file)
        train_gdf = file_gdf[file_gdf['train'] == 1]
        test_gdf = file_gdf[file_gdf['test'] == 1]
        train_test_areas['train_area'] += get_polygon_area(train_gdf)
        train_test_areas['test_area'] += get_polygon_area(test_gdf)
    print(f"Train area: {train_test_areas['train_area']} ha")
    print(f"Test area: {train_test_areas['test_area']} ha")
    print(f"Train percentage: {train_test_areas['train_area'] / (train_test_areas['train_area'] + train_test_areas['test_area']) * 100:.2f}%")
    print(f"Test percentage: {train_test_areas['test_area'] / (train_test_areas['train_area'] + train_test_areas['test_area']) * 100:.2f}%")
    return train_test_areas


In [129]:
als_areas = {'NVIS_8L_fm_mape_r1_KI2020':       [{'date': '30/01/2020', 'area': '2536', 'sensor': 'RIEGL Q680i-S'},
                                                 {'date': '07/02/2020', 'area': '2048', 'sensor': 'RIEGL Q680i-S'},
                                                 {'date': '23/02/2020', 'area': '599', 'sensor': 'RIEGL Q680i-S'},
                                                 {'date': '13/04/2020', 'area': '1985', 'sensor': 'RIEGL Q680i-S'},
                                                 {'date': '14/04/2020', 'area': '3782', 'sensor': 'RIEGL Q680i-S'},
                                                 {'date': '10/09/2020', 'area': '166', 'sensor': 'RIEGL Q680i-S'},
                                                 {'date': '12/09/2020', 'area': '2822', 'sensor': 'RIEGL Q680i-S'}],
             'NVIS_8L_fm_mape_r1_KI2025':       [{'date': '18/06/2025', 'area': '20903', 'sensor': 'RIEGL VUX 160-23'}],
             'NVIS_8L_fm_mape_r1_WASouth':      [{'date': '20/04/2024', 'area': '18659', 'sensor': 'RIEGL VUX 160-23'},
                                                 {'date': '21/04/2024', 'area': '5409', 'sensor': 'RIEGL VUX 160-23'}],
             'NVIS_8L_fm_mape_r1_Billabong':    [{'date': '15/04/2024', 'area': '16153', 'sensor': 'RIEGL VUX 160-23'}],
             'NVIS_8L_fm_mape_r1_MulgaLands':   [{'date': '25/04/2023', 'area': '9518', 'sensor': 'RIEGL Q680i-S'}],
             'NVIS_8L_fm_mape_r1_Tenterfield':  [{'date': '22/05/2025', 'area': '21576', 'sensor': 'RIEGL VUX 160-23'},
                                                 {'date': '23/05/2025', 'area': '21770', 'sensor': 'RIEGL VUX 160-23'}]
}

In [134]:
def get_als_metadata(outputs, polygons):
    # print(len(nn_datasets))
    # print(len(polygons))

    # polygons_names = [os.path.splitext(os.path.basename(x))[0] for x in polygons]
    # # print(polygons_names)
    # filtered_datasets = [x for x in nn_datasets if x.split('/')[9] in polygons_names]
    # # print(len(filtered_datasets))
    # train_datasets = [x for x in filtered_datasets if x.split('/')[7] == 'train']
    # test_datasets = [x for x in filtered_datasets if x.split('/')[7] == 'test']
    # # print(train_datasets)
    # # print(test_datasets)
    # datasets_df = pd.DataFrame({'datasets': filtered_datasets})
    # train_datasets_df = pd.DataFrame({'train_datasets': train_datasets})
    # test_datasets_df = pd.DataFrame({'test_datasets': test_datasets})
    # train_datasets_df['image_tile'] = train_datasets_df['train_datasets'].apply(lambda x: os.path.splitext(os.path.basename(x))[0])
    # test_datasets_df['image_tile'] = test_datasets_df['test_datasets'].apply(lambda x: os.path.splitext(os.path.basename(x))[0])
    # # print(train_datasets_df)
    # # print(test_datasets_df)

    # # print(polygons_names)
    # datasets_df['als_name'] = datasets_df['datasets'].apply(lambda x: x.split('/')[8])
    # datasets_df['als_date'] = datasets_df['als_name'].str.extract(r'^(\d{6})')
    # print(datasets_df['als_date'].unique())

    als_metadata = get_train_test_areas(polygons)
    print(als_metadata)
    als_metadata['captures'] = []
    for als_area in als_areas[outputs]:
        als_metadata['captures'].append(als_area)
        # print(als_area)
    # print(als_areas[outputs])
    return als_metadata

als_metadata_dict = get_als_metadata(outputs, polygons)
print(als_metadata_dict)

Train area: 7498.509053999999 ha
Test area: 1265.7581250000007 ha
Train percentage: 85.56%
Test percentage: 14.44%
{'train_area': 7498.509053999999, 'test_area': 1265.7581250000007}
{'train_area': 7498.509053999999, 'test_area': 1265.7581250000007, 'captures': [{'date': '30/01/2020', 'area': '2536', 'sensor': 'RIEGL Q680i-S'}, {'date': '07/02/2020', 'area': '2048', 'sensor': 'RIEGL Q680i-S'}, {'date': '23/02/2020', 'area': '599', 'sensor': 'RIEGL Q680i-S'}, {'date': '13/04/2020', 'area': '1985', 'sensor': 'RIEGL Q680i-S'}, {'date': '14/04/2020', 'area': '3782', 'sensor': 'RIEGL Q680i-S'}, {'date': '10/09/2020', 'area': '166', 'sensor': 'RIEGL Q680i-S'}, {'date': '12/09/2020', 'area': '2822', 'sensor': 'RIEGL Q680i-S'}]}


/home/tony/miniconda3/envs/qgis_env/lib/python3.9/site-packages/pyogrio/raw.py:198: RuntimeWarning: Several features with id = 1 have been found. Altering it to be unique. This warning will not be emitted anymore for this layer
  return ogr_read(


In [135]:
# geospatial_metadata_dict = get_geospatial_metadata('/mnt/BAMspace3/2_project_specific/client_work/NVIS/NVIS_V7_Validation/Final_Deliverables/Eastern_Australia_Mulga_Shrublands/Eastern_Australia_Mulga_Shrublands_ground_cover.tif')
# print(get_geospatial_metadata('/mnt/BAMspace3/2_project_specific/client_work/NVIS/NVIS_V7_Validation/Final_Deliverables/Eastern_Australia_Mulga_Shrublands/Eastern_Australia_Mulga_Shrublands_middle_cover.tif'))
# print(get_geospatial_metadata('/mnt/BAMspace3/2_project_specific/client_work/NVIS/NVIS_V7_Validation/Final_Deliverables/Eastern_Australia_Mulga_Shrublands/Eastern_Australia_Mulga_Shrublands_upper_cover.tif'))

geospatial_metadata_dict = get_geospatial_metadata('/mnt/datapool1/datapool1/datasets/nvis_outputs/NVIS_8L_fm_mape_r1_KI2020/merged_clipped/NVIS_8L_fm_mape_r1_KI2020_all_merged_clipped_epsg9473.tif')
print(geospatial_metadata_dict)

{'driver': 'GTiff', 'count': 8, 'width': 11646, 'height': 5950, 'dtype': 'float32', 'nodata': -9999.0, 'crs': 'EPSG:9473', 'bounds': {'left': 408458.0397783468, 'bottom': -3953092.1752102603, 'right': 524870.1557080749, 'top': -3893616.639422704}, 'resolution': {'x': 9.995888367656544, 'y': 9.995888367656544}, 'transform': [9.995888367656544, 0.0, 408458.0397783468, 0.0, -9.995888367656544, -3893616.639422704, 0.0, 0.0, 1.0], 'spatial_extent': {'width_meters': 116412.11592972814, 'height_meters': 59475.53578755632, 'area_square_meters': 6923672967.083701}, 'bands': [{'band_number': 1, 'height_range': 'below_0.5m', 'min': 0.0, 'max': 0.8188452124595642, 'mean': 0.04739305799946643, 'std': 0.06404414008407064, 'dtype': 'float32', 'valid_pixels': 40969170, 'total_pixels': 69293700}, {'band_number': 2, 'height_range': '0.5-1m', 'min': 0.0, 'max': 0.6298410296440125, 'mean': 0.046633203089054526, 'std': 0.04077914826477433, 'dtype': 'float32', 'valid_pixels': 40969170, 'total_pixels': 69293

In [115]:
nn_datasets

['/mnt/datapool1/datapool1/datasets/nn_datasets/NVIS/train/250523_A_NSW_2_Record002_7856_chm_blur20_cc8_layer/250523_A_NSW_2_train_test_split/Tenterfield_JL1KF02B01_0007_20250619_1401662_COMBINED_Ortho_Merged.hdf5',
 '/mnt/datapool1/datapool1/datasets/nn_datasets/NVIS/train/250523_A_NSW_2_Record002_7856_chm_blur20_cc8_layer/250523_A_NSW_2_train_test_split/Tenterfield_JL1KF02B01_0008_20250619_1401662_COMBINED_Ortho_Merged.hdf5',
 '/mnt/datapool1/datapool1/datasets/nn_datasets/NVIS/train/240420_Denmark_1_850ft80kn1800khz280lps_chm_blur20_cc8_layer/240420_Denmark_train_test_split/WA_2025_JL1KF02B04_0012_20241211_1336009_COMBINED_Ortho_Merged.hdf5',
 '/mnt/datapool1/datapool1/datasets/nn_datasets/NVIS/train/240420_Denmark_1_850ft80kn1800khz280lps_chm_blur20_cc8_layer/240420_Denmark_train_test_split/WA_2025_JL1KF02B06_0001_20250220_1336017_COMBINED_Ortho_Merged.hdf5',
 '/mnt/datapool1/datapool1/datasets/nn_datasets/NVIS/train/240420_Denmark_1_850ft80kn1800khz280lps_chm_blur20_cc8_layer/2404

In [136]:
clipping_polygons = glob(polygon_path + '/*.geojson')
print(clipping_polygons)
clipping_polygon_names = [os.path.splitext(os.path.basename(f))[0].replace(f'{delivery_name}_', '').replace('KangarooIsland_AdditionalScene_', '') for f in clipping_polygons]
print(clipping_polygon_names)
imagery_metadata_dict = {}
imagery_metadata_dict['mosaic_tiles'] = clipping_polygon_names
print(imagery_metadata_dict)

['/mnt/datapool1/datapool1/datasets/nvis_outputs/NVIS_8L_fm_mape_r1_KI2020/clipping_polygons/Kangaroo_Island2020_JL1KF01A_0031_20201226_1325274_MSS_Ortho.geojson', '/mnt/datapool1/datapool1/datasets/nvis_outputs/NVIS_8L_fm_mape_r1_KI2020/clipping_polygons/Kangaroo_Island2020_JL1KF01A_0032_20201226_1325275_MSS_Ortho.geojson', '/mnt/datapool1/datapool1/datasets/nvis_outputs/NVIS_8L_fm_mape_r1_KI2020/clipping_polygons/KangarooIsland_AdditionalScene_JL1KF01A_0032_20201226_200036508_MSS_Ortho.geojson', '/mnt/datapool1/datapool1/datasets/nvis_outputs/NVIS_8L_fm_mape_r1_KI2020/clipping_polygons/Kangaroo_Island2020_JL1KF01A_0031_20201226_1325656_MSS_Ortho_2.geojson', '/mnt/datapool1/datapool1/datasets/nvis_outputs/NVIS_8L_fm_mape_r1_KI2020/clipping_polygons/Kangaroo_Island2020_JL1KF01A_0007_20220109_1325278_MSS_Ortho.geojson', '/mnt/datapool1/datapool1/datasets/nvis_outputs/NVIS_8L_fm_mape_r1_KI2020/clipping_polygons/Kangaroo_Island2020_JL1GF02A_0005_20201220_1325480_MSS_Ortho_1.geojson', '/mn

In [137]:
# metadata_dict.to_json('/mnt/datapool1/datapool1/datasets/nvis_outputs/{outputs}/metadata.json')
metadata_dict = {
    'geospatial_metadata': geospatial_metadata_dict,
    'als_metadata': als_metadata_dict,
    'imagery_metadata': imagery_metadata_dict
}
xml_bytes = dicttoxml(metadata_dict, custom_root='MetaData', attr_type=False)
print(xml_bytes)
xml_string = parseString(xml_bytes).toprettyxml()
print(xml_string)
with open(f'/mnt/datapool1/datapool1/datasets/nvis_outputs/{outputs}/KI2020_metadata.xml', 'w') as f:
    f.write(xml_string)

b'<?xml version="1.0" encoding="UTF-8" ?><MetaData><geospatial_metadata><driver>GTiff</driver><count>8</count><width>11646</width><height>5950</height><dtype>float32</dtype><nodata>-9999.0</nodata><crs>EPSG:9473</crs><bounds><left>408458.0397783468</left><bottom>-3953092.1752102603</bottom><right>524870.1557080749</right><top>-3893616.639422704</top></bounds><resolution><x>9.995888367656544</x><y>9.995888367656544</y></resolution><transform><item>9.995888367656544</item><item>0.0</item><item>408458.0397783468</item><item>0.0</item><item>-9.995888367656544</item><item>-3893616.639422704</item><item>0.0</item><item>0.0</item><item>1.0</item></transform><spatial_extent><width_meters>116412.11592972814</width_meters><height_meters>59475.53578755632</height_meters><area_square_meters>6923672967.083701</area_square_meters></spatial_extent><bands><item><band_number>1</band_number><height_range>below_0.5m</height_range><min>0.0</min><max>0.8188452124595642</max><mean>0.04739305799946643</mean>